In [1]:
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D, Flatten, Dense, Input, Dropout
from keras.models import Model, load_model
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator


#path = '/home/ubuntu/nbs/state_farm/data/sample1/'
#batch_size=1

path = '/home/ubuntu/nbs/state_farm/data/sample/'
#path = '/home/ubuntu/nbs/state_farm/data/'
batch_size = 64

target_size=(224, 224)

nb_train_samples = 1600
nb_validation_samples = 300
epochs = 50



#generate the batches
def get_batches(directory, target_size=target_size, batch_size=batch_size, shuffle=False):
    datagen = ImageDataGenerator()
    return datagen.flow_from_directory(directory=directory,
                                          target_size=target_size,
                                          batch_size=batch_size,
                                          class_mode='categorical',
                                          shuffle=shuffle)

train_gen = ImageDataGenerator(rotation_range=0, 
                                     width_shift_range=0, 
                                     height_shift_range=0.1, 
                                     shear_range=0.1, 
                                     zoom_range=0.2, 
                                     channel_shift_range=10,
                                     fill_mode='nearest')

batches = train_gen.flow_from_directory(directory=path+'train_set',  
                                        target_size=target_size,
                                        batch_size=batch_size,
                                        class_mode='categorical',
                                        shuffle=True)

#batches = get_batches(path+'train_set', shuffle=True)

valid_batches = get_batches(path+'valid_set', batch_size=batch_size*2, shuffle=False)

# initialize the model
initial_model = VGG16(weights="imagenet", include_top=True)

#finetuning
x = Dense(batches.num_class, activation='softmax')(initial_model.layers[-2].output)
model = Model(initial_model.input, x)

# we freeze the other layers 
for layer in initial_model.layers: layer.trainable=False
print(model.summary())


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


Found 800 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________________________________________________________

In [ ]:
opt = Adam(lr=0.001)

model.compile(optimizer=opt,
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit_generator(batches, epochs=10, 
                    steps_per_epoch=nb_train_samples // batch_size,
                    validation_data=valid_batches, 
                    validation_steps=nb_validation_samples // batch_size)

In [2]:
for layer in model.layers[:10]: 
    layer.trainable = False 

for layer in model.layers[10:]: 
    layer.trainable = True
    
opt = SGD(lr=10e-5) 
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(batches, epochs=10, 
                    steps_per_epoch=nb_train_samples // batch_size,
                    validation_data=valid_batches, 
                    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
25/25 [==============================] - 70s - loss: 0.3311 - acc: 0.8956 - val_loss: 2.1979 - val_acc: 0.4186
Epoch 2/10
25/25 [==============================] - 70s - loss: 0.2635 - acc: 0.9288 - val_loss: 2.0861 - val_acc: 0.4836
Epoch 3/10
25/25 [==============================] - 68s - loss: 0.2213 - acc: 0.9313 - val_loss: 2.1890 - val_acc: 0.4622
Epoch 4/10
25/25 [==============================] - 70s - loss: 0.1740 - acc: 0.9419 - val_loss: 2.2027 - val_acc: 0.4860
Epoch 5/10
25/25 [==============================] - 70s - loss: 0.1643 - acc: 0.9481 - val_loss: 2.3180 - val_acc: 0.4813
Epoch 6/10
25/25 [==============================] - 69s - loss: 0.1267 - acc: 0.9650 - val_loss: 2.4476 - val_acc: 0.4477
Epoch 7/10
25/25 [==============================] - 71s - loss: 0.1260 - acc: 0.9651 - val_loss: 2.2782 - val_acc: 0.5023
Epoch 8/10
25/25 [==============================] - 69s - loss: 0.1201 - acc: 0.9650 - val_loss: 2.3105 - val_acc: 0.4709
Epoch 9/10
25/25 [======

In [6]:
# predictions on new data
test_batches = get_batches(path+'test', batch_size=batch_size)
preds = model.predict_generator(test_batches, steps=test_batches.samples)


Found 10 images belonging to 1 classes.


In [8]:
preds_clipes = preds.clip(min=0.01, max=0.99)
print(preds_clipes[:10])

[[ 0.01        0.03629445  0.01576909  0.01        0.01        0.01        0.01
   0.94735545  0.01        0.01      ]
 [ 0.01        0.01        0.99000001  0.01        0.01        0.01        0.01
   0.01        0.01        0.01      ]
 [ 0.76043767  0.01        0.01        0.01        0.01        0.14135244
   0.01        0.01        0.01        0.09578142]
 [ 0.01        0.15547365  0.76144546  0.01        0.01        0.01
   0.01636479  0.01        0.04142869  0.02074409]
 [ 0.01        0.01        0.01        0.01        0.01        0.01        0.01
   0.9820087   0.01433661  0.01      ]
 [ 0.01        0.01        0.01        0.01        0.01        0.57158494
   0.04882823  0.01700319  0.27842256  0.07719808]
 [ 0.99000001  0.01        0.01        0.01        0.01        0.01        0.01
   0.01        0.01        0.01      ]
 [ 0.01639338  0.01        0.01        0.38630763  0.01        0.5032298
   0.02658342  0.01        0.04396535  0.01841919]
 [ 0.02812427  0.01        0.01

In [ ]:
model.save('models/data_augmentation.h5')